In [1]:
import math
from collections import namedtuple

In [2]:
Customer = namedtuple("Customer", ['index', 'demand', 'x', 'y'])

In [3]:
file_location = "../data/vrp_5_4_1"
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()

lines = input_data.split('\n')
parts = lines[0].split()
customer_count = int(parts[0])
vehicle_count = int(parts[1])
vehicle_capacity = int(parts[2])
    
customers = []
for i in range(1, customer_count+1):
    line = lines[i]
    parts = line.split()
    customers.append(Customer(i-1, int(parts[0]), float(parts[1]), float(parts[2])))

#the depot is always the first customer in the input
depot = customers[0] 


In [4]:
def dist(customer1, customer2):
    return math.sqrt((customer1.x - customer2.x)**2 + (customer1.y - customer2.y)**2)

In [5]:
print (customers)
print("---------------")
print(depot)

[Customer(index=0, demand=0, x=0.0, y=0.0), Customer(index=1, demand=3, x=0.0, y=10.0), Customer(index=2, demand=3, x=-10.0, y=10.0), Customer(index=3, demand=3, x=0.0, y=-10.0), Customer(index=4, demand=3, x=10.0, y=-10.0)]
---------------
Customer(index=0, demand=0, x=0.0, y=0.0)


In [6]:
def obj(vehicle_tours, ):
    obj = 0
    for v in range(0, vehicle_count):
        vehicle_tour = vehicle_tours[v]
        if len(vehicle_tour) > 0:
            obj += dist(depot,vehicle_tour[0])
            for i in range(0, len(vehicle_tour)-1):
                obj += dist(vehicle_tour[i],vehicle_tour[i+1])
            obj += dist(vehicle_tour[-1],depot)
    return obj

In [7]:
vehicle_tours = []
vehicle_tours = [[] for _ in range(vehicle_count)]
remaining_customers = set(customers)
remaining_customers.remove(depot)
vehicle_loads = [0 for _ in range(vehicle_count)]

while len(remaining_customers) > 0:
    customer = remaining_customers.pop()
    for i in range(vehicle_count):
        if vehicle_loads[i] + customer.demand <= vehicle_capacity:
            vehicle_tours[i].append(customer)
            vehicle_loads[i] += customer.demand
            break

In [8]:
remaining_customers

set()

In [9]:
vehicle_loads

[9, 3, 0, 0]

In [10]:
vehicle_tours

[[Customer(index=4, demand=3, x=10.0, y=-10.0),
  Customer(index=1, demand=3, x=0.0, y=10.0),
  Customer(index=2, demand=3, x=-10.0, y=10.0)],
 [Customer(index=3, demand=3, x=0.0, y=-10.0)],
 [],
 []]

In [12]:
obj(vehicle_tours)

80.6449510224598